In [1]:
!pip install -U spacy
!pip install -U spacy-looups-data
!pip install -m spacy download en_core_web_sm
!pip install -m spacy download en_core_web_md
!pip install -m spacy download en_core_web_lg

Requirement already up-to-date: spacy in f:\anaconda3\lib\site-packages (2.2.4)


  Could not find a version that satisfies the requirement spacy-looups-data (from versions: )
No matching distribution found for spacy-looups-data

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -m

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -m

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options]

## General Feature Extraction

- File Loading
- Word Counts
- Charecter Counts
- Average Charecters Per Word
- Stop Words Count
- Count #HashTags and @Mentions
- If numeric digits are present in the tweet
- Uppercase word count

In [2]:
# File loading
import pandas as pd
import numpy as np

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
df = pd.read_csv("A:/training.1600000.processed.noemoticon.csv", encoding = "latin1", header = None)

In [3]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df.shape

# The data has 1.6 million rows and 6 columns

(1600000, 6)

In [5]:
df = df[[5,0]]

In [6]:
df.columns = ["tweets", "sentiments"]

In [7]:
df.head()

,tweets,sentiments
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [8]:
# Calculating the value counts of sentiments

df["sentiments"].value_counts()

4    800000
0    800000
Name: sentiments, dtype: int64

In [9]:
# The sentiments columns has 800000 positive tweets and 800000 negative tweets

In [10]:
sent_map = { 0 : "negative", 4 : "positive"}

### Word Count

In [11]:
df["word_count"] = df["tweets"].apply(lambda x : len(str(x).split()))
                   
# We have used str(x) coz in tweets there can be some numeric charecters which cannot be split. 
# Hence we converted all the charecters to string                  

- The split() method splits a string into a list.

- You can specify the separator, default separator is any whitespace.

In [12]:
df.head()

,tweets,sentiments,word_count
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19
1,is upset that he can't update his Facebook by ...,0,21
2,@Kenichan I dived many times for the ball. Man...,0,18
3,my whole body feels itchy and like its on fire,0,10
4,"@nationwideclass no, it's not behaving at all....",0,21


### Charecter Count 

In [13]:
df["charecter_count"] = df["tweets"].apply(lambda x : len(x))

In [14]:
df.head()

,tweets,sentiments,word_count,charecter_count
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115
1,is upset that he can't update his Facebook by ...,0,21,111
2,@Kenichan I dived many times for the ball. Man...,0,18,89
3,my whole body feels itchy and like its on fire,0,10,47
4,"@nationwideclass no, it's not behaving at all....",0,21,111


### Calculating Average length of each word

In [15]:
def get_avg_word_len(x):
    words = x.split()
    word_len = 0
    for word in words:
        word_len = word_len + len(word)
    return word_len/len(words)        

- We have created a function **get_avg_word_len**
- It first divides the sentence x into a list of words which is stored in variable called **words**
- We set a variable word_len = 0
- We pass a for loop which takes each word and calculates its length
- Finally, it returns length of the word divided bt **total number** of words in the list **words** 

In [16]:
df ["avg_word_length"] = df["tweets"].apply(lambda x : get_avg_word_len(x))

In [17]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444
3,my whole body feels itchy and like its on fire,0,10,47,3.700000
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714


- we can also calculate avg_word_length = charecter_count/word_count
- But we are creating a function for our calculations because, in charecter_count, it has also calculated the white_spaces between the charecters

### Calculating STOP WORDS

In [18]:
print(STOP_WORDS)

{'alone', 'forty', 'using', 'on', 'noone', 'it', 'either', 'otherwise', 'nine', '’ll', 'indeed', 'before', 'thru', 'once', 'more', 'now', 'us', 'somewhere', 'our', 'well', 'because', 'about', 'ca', 'make', 'over', 'again', 'above', 'be', 'another', 'have', 'around', 'thereupon', 'unless', 'part', 'many', 'wherein', 'formerly', 'via', 'per', 'seems', 'been', 'these', 'never', 'hers', 'already', 'could', 'out', 'throughout', 'through', 'some', 'seeming', 'hence', 'toward', "'m", 'say', 'nothing', 'amongst', 'only', 'so', 'no', 'used', 'hereafter', 'has', 'any', 'most', 'why', 'anyone', 'too', 'upon', 'even', 'being', 'whither', 'several', 'sixty', 'quite', 'whose', 'when', 'towards', 'move', "'ve", 'top', 'under', 'whom', 'here', 'together', "'s", '‘ve', 'herein', 'almost', 'due', 'is', 'a', 'yourself', 'she', 'always', 'then', 'into', '‘d', 'i', 'however', '‘re', 'can', 'fifteen', 'one', 'or', 'whereas', 'such', 'which', 'onto', 'five', 'yourselves', 'how', 'ourselves', 'fifty', 'bottom

- Most of the times we try to remove the stop words
- But in some applications, stop_words hold alot of meaning
- If we are trying to find the context meaning of the sentence, like we are trying to find what this sentence is saying. That time, stop words become very useful.

In [19]:
df["stop_word_count"] = df["tweets"].apply(lambda x : len([t for t in x.split() if t in STOP_WORDS]))

- In lambda function, we get a list of stop words and then we calculate the length of that list

In [20]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10


### Calculate Number of #tags and @Mentions present in each tweet

In [21]:
# lets see an exaple
x = "This is #Hashtag and this is @Mention"
[t for t in x.split() if t.startswith("#") or t.startswith("@")]

['#Hashtag', '@Mention']

In [22]:
df["hashtags"] = df["tweets"].apply(lambda x: len([t for t in x.split() if t.startswith("#")]))

In [23]:
df["mentions"] = df["tweets"].apply(lambda x: len([t for t in x.split() if t.startswith("@")]))

In [ ]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,mentions
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4,0,1
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9,0,0
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7,0,1
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10,0,1


In [ ]:
df = df.drop("mentions", axis = 1)

### Check If Numeric data is present in the tweet

In [ ]:
df["numeric_count"] = df["tweets"].apply(lambda x: len([t for t in x.split() if t.isdigit()]))

In [ ]:
df.head(10)

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4,0,0
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9,0,0
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7,0,0
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10,0,0
5,@Kwesidei not the whole crew,0,5,29,4.800000,3,0,0
6,Need a hug,0,3,11,2.666667,1,0,0
7,@LOLTrish hey long time no see! Yes.. Rains a...,0,22,99,3.454545,4,0,0
8,@Tatiana_K nope they didn't have it,0,6,36,5.000000,3,0,0
9,@twittera que me muera ?,0,5,25,4.000000,1,0,0


### UPPER case word count.

- Upper case word count is very important as people often describe their happiness and Rage in Upper case to emphasize the words.

In [ ]:
df["upper_case_words"] = df["tweets"].apply(lambda x: len([t for t in x.split() if t.isupper() and len(x) > 2]))

- We used len(x) > 2 coz some words like "I", "AM" are written in upper case but do not show any sentiment

In [ ]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4,0,0,1
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9,0,0,0
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7,0,0,1
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10,0,0,1


In [ ]:
# lets see some tweets
df.loc[96, "tweets"]

"so rylee,grace...wana go steve's party or not?? SADLY SINCE ITS EASTER I WNT B ABLE 2 DO MUCH  BUT OHH WELL....."

## Data Preprocessing and Cleaning

### Preprocessing and Cleaning

- Lower case
- Contraction to Expansion
- Emails removal and counts
- URLs removal and counts
- Removal of RT
- Removal of Special Characters
- Removal of multiple spaces
- Removal of HTML tags
- Removal of accented characters
- Removal of Stop Words
- Conversion into base form of words
- Common Occuring words Removal
- Rare Occuring words Removal
- Word Cloud
- Spelling Correction
- Tokenization
- Lemmatization
- Detecting Entities using NER
- Noun Detection
- Language Detection
- Sentence Translation
- Using Inbuilt Sentiment Classifier

### 1. Converting all charecters to lower case

In [ ]:
df["tweets"] = df["tweets"].apply(lambda x : x.lower())

In [ ]:
df.head(3)

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words
0,"@switchfoot http://twitpic.com/2y1zl - awww, t...",0,19,115,5.052632,4,0,0,1
1,is upset that he can't update his facebook by ...,0,21,111,4.285714,9,0,0,0
2,@kenichan i dived many times for the ball. man...,0,18,89,3.944444,7,0,0,1


### 2. Contraction to Expansion

- Contraction means shortform of words. For eg: [don't, haven't, wouldn't, he'll, i'd]
- Expansion means fullform of those words: [do not, have not, would not, he will, i would]
- Contraction to Expansion is necessary to understand the meaning of each sentence.                                                 

In [ ]:
# Creating a list of contractions from Wikipedia

contractions = {"aight" : "alright",
 "ain't": "am not",
 "amn't" : "am not",
 "aren't": "are not",
 "can't": "cannot",
 "'cause" : "because",
 "could've": "could have",
 "couldn't" : "could not",
 "couldn't've" : "could not have", 
 "daren't" : "dare not",
 "daresn't" : "dare not",
 "dasn't" : "dare not",
 "didn't" : "did not",
 "doesn't" : "does not",
 "don't" : "do not",
 "d'ye" : "do you",
 "e'er" : "ever",
 "everybody's" : "everybody is",
 "everyone's" : "everyone is",
 "finna":"fixing to",
 "g'day" : "good day",
 "gimme" : "give me",
 "giv'n": "given",
 "gonna":"going to",
 "gon't":"go not",
 "gotta":"got to",
 "hadn't":"had not",
 "had've":"had have",
 "hasn't":"has not",
 "haven't":"have not",
 "he'd":"he would",
 "he'dn't've'd":"he would not have had",
 "he'll":"he will",
 "he's":"he is",
 "he've":"he have",
 "how'd":"how did",
 "howdy":"how do you do",
 "how'll":"how will",
 "how're":"how are",
 "how's":"how has",
 "i'd": "I would",
 "i'd've":"I would have",
 "i'll": "I will",
 "i'm": "I am",
 "i'm'a": "I am about to",
 "i'm'o": "I am going to",
 "innit": "is it not",
 "i've": "I have",
 "isn't": "is not",
 "it'd": "it would",
 "it'll": "it will",
 "it's": "it has",
 "let's": "let us", 
 "ma'am": "madam",
 "mayn't": "may not",
 "may've": "may have",
 "methinks" : "me thinks",
 "mightn't": "might not",
 "might've": "might have",
 "mustn't": "must not",
 "mustn't've": "must not have",
 "must've": "must have",
 "needn't": "need not",
 "ne'er":"never",
 "o'clock": "of the clock",
 "o'er": "over",
 "ol'": "old",
 "oughtn't":"ought not",
 "'s": "is",
 "shalln't":"shall not",
 "shan't":"shall not",
 "she'd":"she would",
 "she'll":"she will",
 "she's":"she is",
 "should've":"should have",
 "shouldn't":"should not",
 "shouldn't've":"should not have",
 "somebody's":"somebody is",
 "someone's":"someone is",
 "something's":"something is",
 "so're":"so you are",
 "that'll":"that will",
 "that're":"that are",
 "that's":"that is",
 "that'd":"that had",
 "there'd":"there would",
 "there'll":"here shall",
 "there're":"there are",
 "there's":"there has",
 "these're":"these are",
 "these've":"these have",
 "they'd":"they would",
 "they'll":"they will",
 "they're":"they are",
 "they've":"they have",
 "this's":"this is",
 "those're":"those are",
 "those've":"those have",
 "'tis":"it is",
 "to've":"to have",
 "'twas":"it was",
 "wanna":"want to",
 "wasn't":"was not",
 "we'd":"we would",
 "we'd've":"we would have",
 "we'll":"we will",
 "we're":"we are",
 "we've":"we have",
 "weren't":"were not",
 "what'd":"what did",
 "what'll":"what will",
 "what're":"what are",
 "what's":"what is",
 "what've":"what have",
 "when's":"when is",
 "where'd":"where did",
 "where'll":"where will",
 "where're":"where are",
 "where's":"where has",
 "where've":"where have",
 "which'd":"which had",
 "which'll":"which shall",
 "which're":"which are",
 "which's":"which has",
 "which've":"which have",
 "who'd":"who would",
 "who'd've":"who would have",
 "who'll":"who will",
 "who're":"who are",
 "who's":"who has",
 "who've":"who have",
 "why'd":"why did",
 "why're":"why are",
 "why's":"why is",
 "won't":"will not",
 "would've":"would have",
 "wouldn't":"would not",
 "wouldn't've":"would not have",
 "y'all":"you all",
 "y'all'd've":"you all would have",
 "y'all'dn't've'd":"you all would not have had",
 "y'all're":"you all are",
 "you'd":"you would",
 "you'll":"you will",
 "you're":"you are",
 "you've":"you have",
  " u " : " you",
 " ur " : " your",
 " n ": " and"}

In [ ]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key,value)
        return x
    else:
        return x

In [ ]:
%%time
df["tweets"] = df["tweets"].apply(lambda x: cont_to_exp(x))

Wall time: 4min 15s


In [ ]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words
0,"@switchfoot http://twitpic.com/2y1zl - awww, t...",0,19,115,5.052632,4,0,0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0
2,@kenichan i dived many times for the ball. man...,0,18,89,3.944444,7,0,0,1
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0
4,"@nationwideclass no, it has not behaving at al...",0,21,111,4.285714,10,0,0,1


### Count and Remove Email ID

In [ ]:
import re

In [ ]:
re.findall(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9,_-]+\.[a-zA_Z0-9._-]+)', x)

[]

- in re.findall and re.sub (r' means remove)

In [ ]:
df["emails"] = df["tweets"].apply(lambda x : re.findall(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9,_-]+\.[a-zA_Z0-9._-]+)', x))

In [ ]:
df["email_counts"] = df["emails"].apply(lambda x : len(x))

In [ ]:
df[df["email_counts"]>0]

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words,emails,email_counts
4054,i want a new laptop. hp tx2000 is the bomb. :...,0,20,103,4.150000,6,0,0,4,[gabbehhramos@yahoo.com],1
7917,who stole elledell@gmail.com?,0,3,31,9.000000,1,0,0,0,[elledell@gmail.com],1
8496,@alexistehpom really? did you send out all th...,0,20,130,5.500000,11,0,0,0,[missataari@gmail.com],1
10290,@laureystack awh...thatis kinda sad lol add m...,0,8,76,8.500000,0,0,0,0,[hello.kitty.65@hotmail.com],1
16413,"@jilliancyork got 2 bottom of it, human error...",0,21,137,5.428571,7,0,1,0,[press@linkedin.com],1
61735,"sumthin, younoe how sum girls are lyk an hourg...",0,20,129,5.450000,6,0,0,0,[83w@r3...],1
65407,"@suzesinc, rb ...run out of ya allocated props...",0,15,136,8.000000,3,0,0,2,[2fast4u@s...],1
72386,"madonnafae ok, i found this on a fluff page~ e...",0,22,136,5.000000,9,0,0,0,[nrqj8ga@facebook.com],1
74295,@lovetoblog i cannot dm you my email cos you a...,0,16,105,5.562500,6,0,0,2,[danielleloren@live.co.uk],1
87268,but...@floppyflopsome...cannot you come for ju...,0,13,89,5.846154,6,0,0,1,[but...@floppyflopsome...cannot],1


In [ ]:
# removing emails from tweets using re.sub [re.substitute]

df["tweets"] = df["tweets"].apply(lambda x : re.sub(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9,_-]+\.[a-zA_Z0-9._-]+)', "", x))

In [ ]:
df[df["email_counts"]>0]

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words,emails,email_counts
4054,i want a new laptop. hp tx2000 is the bomb. :...,0,20,103,4.150000,6,0,0,4,[gabbehhramos@yahoo.com],1
7917,who stole ?,0,3,31,9.000000,1,0,0,0,[elledell@gmail.com],1
8496,@alexistehpom really? did you send out all th...,0,20,130,5.500000,11,0,0,0,[missataari@gmail.com],1
10290,@laureystack awh...thatis kinda sad lol add m...,0,8,76,8.500000,0,0,0,0,[hello.kitty.65@hotmail.com],1
16413,"@jilliancyork got 2 bottom of it, human error...",0,21,137,5.428571,7,0,1,0,[press@linkedin.com],1
61735,"sumthin, younoe how sum girls are lyk an hourg...",0,20,129,5.450000,6,0,0,0,[83w@r3...],1
65407,"@suzesinc, rb ...run out of ya allocated props...",0,15,136,8.000000,3,0,0,2,[2fast4u@s...],1
72386,"madonnafae ok, i found this on a fluff page~ e...",0,22,136,5.000000,9,0,0,0,[nrqj8ga@facebook.com],1
74295,@lovetoblog i cannot dm you my email cos you a...,0,16,105,5.562500,6,0,0,2,[danielleloren@live.co.uk],1
87268,you come for just one weekend? ps. i am so c...,0,13,89,5.846154,6,0,0,1,[but...@floppyflopsome...cannot],1


In [ ]:
df.loc[0,"tweets"]

'@switchfoot http://twitpic.com/2y1zl - awww, thatis a bummer.  you shoulda got david carr of third day to do it. ;d'

### Count URLs and Remove It

In [ ]:
x = " my name is https://mehta.com"

In [ ]:
re.findall(r'(https|ftp|http)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^+%&?~+#-])?', x)

[('https', 'mehta.com', '')]

In [ ]:
df["url_flags"] = df["tweets"].apply(lambda x: len(re.findall(r'(https|ftp|http)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^+%&?~+#-])?', x)))

In [ ]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words,emails,email_counts,url_flags
0,"@switchfoot http://twitpic.com/2y1zl - awww, t...",0,19,115,5.052632,4,0,0,1,[],0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0,[],0,0
2,@kenichan i dived many times for the ball. man...,0,18,89,3.944444,7,0,0,1,[],0,0
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,[],0,0
4,"@nationwideclass no, it has not behaving at al...",0,21,111,4.285714,10,0,0,1,[],0,0


In [ ]:
df["tweets"] = df["tweets"].apply(lambda x: re.sub(r'(https|ftp|http)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^+%&?~+#-])?',"", x))

In [ ]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words,emails,email_counts,url_flags
0,"@switchfoot - awww, thatis a bummer. you sho...",0,19,115,5.052632,4,0,0,1,[],0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0,[],0,0
2,@kenichan i dived many times for the ball. man...,0,18,89,3.944444,7,0,0,1,[],0,0
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,[],0,0
4,"@nationwideclass no, it has not behaving at al...",0,21,111,4.285714,10,0,0,1,[],0,0


In [ ]:
df.loc[0,"tweets"]

'@switchfoot  - awww, thatis a bummer.  you shoulda got david carr of third day to do it. ;d'

### Remove RT (Re-tweet)

In [ ]:
df["tweets"] = df["tweets"].apply(lambda x : re.sub("RT", "", x))

### Removing Special Charecters or Punctuations

- We will only keep alphabetical charecters and numeric charecters.

In [ ]:
import re
df["tweets"] = df["tweets"].apply(lambda x:re.sub('[^A-Z a-z 0-9]+', '', x))

In [ ]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words,emails,email_counts,url_flags
0,switchfoot awww thatis a bummer you shoulda...,0,19,115,5.052632,4,0,0,1,[],0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0,[],0,0
2,kenichan i dived many times for the ball manag...,0,18,89,3.944444,7,0,0,1,[],0,0
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,[],0,0
4,nationwideclass no it has not behaving at all ...,0,21,111,4.285714,10,0,0,1,[],0,0


In [ ]:
df.loc[0, "tweets"]

'switchfoot   awww thatis a bummer  you shoulda got david carr of third day to do it d'

### Removing multiple spaces between words `"Hello     Ashish  "       `

In [ ]:
x = "Thanks    for watching    this video."

In [ ]:
" ".join(x.split())

'Thanks for watching this video.'

In [ ]:
df["tweets"] = df["tweets"].apply(lambda x : " ".join(x.split()))

In [ ]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words,emails,email_counts,url_flags
0,switchfoot awww thatis a bummer you shoulda go...,0,19,115,5.052632,4,0,0,1,[],0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0,[],0,0
2,kenichan i dived many times for the ball manag...,0,18,89,3.944444,7,0,0,1,[],0,0
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,[],0,0
4,nationwideclass no it has not behaving at all ...,0,21,111,4.285714,10,0,0,1,[],0,0


### Removing HTML tags [if any]

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
x = "<html><h2>Thanks for watching</h2></html>"

In [ ]:
BeautifulSoup(x, "lxml").get_text()

'Thanks for watching'

In [ ]:
df["tweets"] = df["tweets"].apply(lambda x: BeautifulSoup(x, "lxml").get_text())

### Removing Accented Charecters

- Accented charecters are words written with decorated characters
- eg: résumé. Look at the e written. The meanings of these words differ slightly from actual english word.

In [ ]:
import unicodedata

def remove_accented_chars(x):
    x = unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8", "ignore")
    return(x)

In [ ]:
df["tweets"] = df["tweets"].apply(lambda x : remove_accented_chars(x))

### Using Spacy and NLP

In [ ]:
import spacy

In [ ]:
x = "this is a stop word code"

In [ ]:
" ".join(t for t in x.split() if t not in STOP_WORDS)

In [ ]:
df["tweets"] = df["tweets"].apply(lambda x : " ".join([t for t in x.split() if t not in STOP_WORDS]))

In [ ]:
df.head()

### Convert into base or root form of word

- It is necessary to convert all the words into base form coz we have to decrease the length of our vocabulary.
- if we dont convert words to base form, then they will be treated as different words.
- Eg: times and time will be treated as different words even though they mean the same.


- Should we convert words to its base form or not depends on the Project that we are working on.

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
x = "kenichan dived timeses flies highs and lows of lives"

In [ ]:
# Base forms are as follows:
# dived = dive, times= time

In [ ]:
def base_form(x):
    x_list = []
    doc = nlp(x)
    
    for token in doc:
        lemma = str(token.lemma_)
        if lemma == "-PRON-" or lemma == "be":
            lemma = token.text
        x_list.append(lemma)
    print(" ".join(x_list))

In [ ]:
base_form(x)

### Common Words Removal

- We need to remove common words because, if the word occurs more number of times, then it means it is not important. [Hypothesis]
- It might vary from project to project

In [ ]:
df.head()["tweets"]

In [ ]:
# 1. Join all the tweets together 

text = " ".join(df["tweets"])

In [ ]:
#2. Create a list of all the words in the joint tweets

text = text.split()

In [ ]:
# 3. Convert text into a column and then count the values of each word

freq_comm = pd.Series(text).value_counts()

In [ ]:
freq_comm

In [ ]:
# Lets say we have to remove top 20 occuring words

f20 = freq_comm[:20]

In [ ]:
f20

In [ ]:
# Remove most common 20 words

df["tweets"] = df["tweets"].apply(lambda x : " ".join(t for t in x.split() if t not in f20))

# DID NOT EXECUTE IT

### Removing the Rare words

In [ ]:
freq_comm

In [ ]:
rare20 = freq_comm[:-20]

### OR

- We can select all the words which have appeared only once as rare words.

In [ ]:
rare = freq_comm[freq_comm.values == 1]

In [ ]:
rare

In [ ]:
df["tweets"] = df["tweets"].apply(lambda x : " ".join([t for t in x.split() if t not in rare20]))

# DID NOT EXECUTE

### WORD CLOUD VISUALIZATION

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
#Taking first 1000 words from the "text" list and converting it to a paragraph

x = " ".join(text[:1000])

In [ ]:
x

In [ ]:
wc = WordCloud(width = 600, height = 500).generate(x)
plt.imshow(wc)
plt.axis("off")
plt.show()

### Spelling Correction

- for spelling corrections, we make use of textblob

In [ ]:
# Install textblob if not installed

#!pip install -U textblob
#!python -m textblob.download_corpora 

In [1]:
from textblob import TextBlob

In [2]:
x = "Tanks forr waching thes vidio carr"

In [3]:
x = TextBlob(x).correct()

In [4]:
x

TextBlob("Ranks for watching the video care")

- We can see that textBlob is not very accurate. We wanted to have Thanks instead of Tanks but it gave Ranks

### Tokenization

- Tokenization means converting text to tokens (single words)

In [ ]:
x = "Thanks# for watching this video. Please Subscribe!"

In [ ]:
TextBlob(x).words

### OR

- We can do Tokenization using Spacy as well

In [ ]:
doc = nlp(x)
for token in doc:
    print(token)

- WordBlob does not consider punctuations and special charecters (# in above sent) as tokens
- Spcay takes punctuations as tokens
- When to use which tokenization technique will depend on our project requirement 

### Lemmatization

In [ ]:
# Lemmatization using textblob

In [ ]:
x = "runs running run ran"

In [ ]:
from textblob import Word

In [ ]:
for token in x:
    print(Word(token).lemmatize())

In [ ]:
# Lemmatization using Spacy

In [ ]:
doc = nlp(x)
for token in doc:
    print(token.lemma_)

- Spacy has a better tokenizer than Wordblob

In [110]:
df.head()

,tweets,sentiments,word_count,charecter_count,avg_word_length,stop_word_count,hashtags,numeric_count,upper_case_words,emails,email_counts,url_flags
0,,0,19,115,5.052632,4,0,0,1,[],0,1
1,,0,21,111,4.285714,9,0,0,0,[],0,0
2,,0,18,89,3.944444,7,0,0,1,[],0,0
3,,0,10,47,3.700000,5,0,0,0,[],0,0
4,,0,21,111,4.285714,10,0,0,1,[],0,0
